# Black Magic AI Detectron2 Object Detection Cloud Vision API Tutorial

<img src="../images/blackmagicailogo.png">

This tutorial demonstrats how to create an AWS Detectron2 Object detection Cloud API by deploying a pre-trained  Detectron2 model to an AWS Sagemaker endpoint and exposing it as a REST API using AWS API Gateway.

You can make a copy of this tutorial by "File -> Open in playground mode" and make changes there. __DO NOT__ request access to this tutorial.


## 1. Install detectron2

In [ ]:
# Versions: https://github.com/pytorch/vision/
# This is the current pytorch version on Colab. Uncomment this if Colab changes its pytorch version
!pip install torch==1.10.2+cu113 torchvision==0.11.3+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html    
# Install detectron2 that matches the above pytorch version
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
!pip install detectron2==0.6 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu113/torch1.10/index.html #commented
exit(0)  # After installation, you need to "restart runtime" in Colab. This line can also restart runtime

In [ ]:
# import some common libraries
import numpy as np
import cv2, json
import torch, torchvision
# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.modeling import build_model
import detectron2.data.transforms as T
from detectron2.checkpoint import DetectionCheckpointer

In [ ]:
# check pytorch installation: 
import sys
print(torch.__version__, torch.cuda.is_available())
print(torchvision.__version__)
print(sys.version_info)

## 2. Run a pre-trained Detectron2 Object Detection model

**Define source Image**

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image, ImageDraw, ImageFont

image_filename="city-scene.jpg"
input_image="../images/" + image_filename

image_src = Image.open(input_image)
np_image = np.array(image_src, dtype='float32')

image_src.show()

**Then, we create a detectron2 config and a detectron2 `DefaultPredictor` to run inference on this image.**

In [ ]:
# Step 1
cfg = get_cfg()

## 3. Build Object Detection Model
["...object detection, where the goal is to classify individual objects and localize them using a bounding box..."](https://kharshit.github.io/blog/2019/08/23/quick-intro-to-instance-segmentation)

**Define custom object detection visualizer**

In [ ]:
def object_visualizer(image_src, predictions, classes, api=False):
    # Draw boxes
    # predictions
    if (api):
        boxes = predictions[0]['pred_boxes']
        pred_classes = predictions[0]['pred_classes']
        scores = predictions[0]['scores']        
    else:               
        boxes = predictions[0]['instances'].pred_boxes
        pred_classes =predictions[0]['instances'].pred_classes
        scores = predictions[0]['instances'].scores

    font = ImageFont.truetype('FreeSerif.ttf', 8)
    draw = ImageDraw.Draw(image_src)
    for box, cl, score in zip(boxes,pred_classes, scores): 
        text = f"{classes[cl]} {score:.0%}"
        len = draw.textlength(text=text)
        bbox = draw.textbbox((box[0], box[1]), text, font=font)
        h = bbox[3] - bbox[1]
   
        draw.rectangle([(box[0], box[1]-h), (box[0] + len, box[1])], fill=(0,0,0))#text background rectangle
        draw.text((box[0], box[1]-h), text, fill=(255, 255, 255))
        draw.rectangle([(box[0], box[1]), (box[2], box[3])], outline=(0,255,0))#blue rectangle
    return image_src

all_classed_list = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

In [ ]:
# Object Detection
# add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7  # set https://object-detection-1.notebook.us-east-2.sagemaker.aws/notebooks/Detectron2_Model_Build_Deploy.ipynb#threshold for this model
# Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml")
model = build_model(cfg)
checkpointer = DetectionCheckpointer(model)
checkpointer.load(cfg.MODEL.WEIGHTS)
model.eval()

**Use Default Predictor and Visualizer to validate Model**

In [ ]:
# Object Detection Visualizer
predictor = DefaultPredictor(cfg) # normal operation
predictions = predictor(np_image)["instances"]
# We can use `Visualizer` to draw the predictions on the image.
v = Visualizer(np_image[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_instance_predictions(predictions.to("cpu"))
plt.imshow(cv2.cvtColor(out.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB))
plt.show()

## 4. Export Model

In [ ]:
# Export mdoel
# Ref: https://pytorch.org/tutorials/beginner/saving_loading_models.html
# https://pytorch.org/vision/stable/index.html
# https://pytorch.org/hub/
# https://github.com/facebookresearch/detectron2/blob/main/MODEL_ZOO.md
# tar cmd: tar -czvf model.tar.gz model
# tar -czvf model.tar.gz model.pth code
torch.save(model, "models/model-object.pth", _use_new_zipfile_serialization=True)

In [ ]:
%%bash
# Create model asset required for Sagemaker endpoint deployment. Copy model.tar.gz to S3 bucket model folder.
# Rename model-object.pth file name to model-object.pth per required by Sagemaker endpoint specs
tar --transform='flags=r;s|models/model-object.pth|model.pth|' -czvf models/model.tar-object.gz models/model-object.pth code/inference.py

**Load Exported Model and Validate**

In [ ]:
saved_object_model = torch.load("models/model-object.pth")
saved_object_model.eval()

In [ ]:
# Model experiment without Detectron2 predictor
original_image = cv2.imread(input_image) 
aug = T.ResizeShortestEdge(
             [800, 800], 1333
#             [cfg.INPUT.MIN_SIZE_TEST, cfg.INPUT.MIN_SIZE_TEST], cfg.INPUT.MAX_SIZE_TEST
        )
with torch.no_grad():  # https://github.com/sphinx-doc/sphinx/issues/4258
        height, width = original_image.shape[:2]
        image = aug.get_transform(original_image).apply_image(original_image)
        image = torch.as_tensor(image.astype("float32").transpose(2, 0, 1))

        inputs = {"image": image, "height": height, "width": width}
        predictions = saved_object_model([inputs])
#         print(predictions)

## 5. Use Detectron2 Visualizer on saved model output

In [ ]:
# We can use `Visualizer` to draw the predictions on the image.
prediction_output=predictions[0]["instances"].to("cpu")
v = Visualizer(np_image[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_instance_predictions(prediction_output)
plt.imshow(cv2.cvtColor(out.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB))
plt.show()

**Display results using custom visualizer which does not use Detectron2 dependances**

In [ ]:
im_out = object_visualizer(image_src, predictions, all_classed_list, False)
im_out.show()

## 6. Deploy Object Detection Model to Endpoint

**Upload model.tar.gz file to s3 bucket model folder**

In [ ]:
import boto3
import sagemaker
from sagemaker.pytorch import PyTorchModel
from sagemaker.predictor import Predictor
from sagemaker import get_execution_role, Session

sess = Session(default_bucket='<INSERT-AWS-S3-BUCKET-NAME-HERE>')

role = get_execution_role()

# Connect to S3 bucket and upload file to s3 bucket
s3 = boto3.resource('s3')
s3.Bucket('<INSERT-AWS-S3-BUCKET-NAME-HERE>').upload_file("models/model.tar-object.gz", "model/model.tar.gz")

uri = sess.list_s3_files(sess.default_bucket(), 'model')
model_data = sagemaker.s3.s3_path_join('s3://', sess.default_bucket(), uri[1])

**Create Pytorch model and deploy to SageMaker endpoint**

In [ ]:
# Object detection
region = sess.boto_region_name
serve_image_uri = f"<INSERT-AWS-ELASTIC-CONTAINER-REGISTRY-REPOSITORY-NAME-HERE>" ##custom image

pyModel = PyTorchModel(
    entry_point="inference.py",
    source_dir="code",
    role=role,
    model_data=model_data,
    image_uri=serve_image_uri,
    framework_version="1.10.2",
    py_version="py38"
)

predictorEndpt = pyModel.deploy(instance_type='ml.p3.2xlarge', initial_instance_count=1)

**Validate Endpoint - perform inference**

In [ ]:
# Ref:
# https://aws.amazon.com/blogs/compute/handling-binary-data-using-amazon-api-gateway-http-apis/
# https://docs.aws.amazon.com/apigateway/latest/developerguide/api-gateway-payload-encodings.html
# https://aws.amazon.com/premiumsupport/knowledge-center/api-gateway-binary-data-lambda/
# https://docs.aws.amazon.com/apigateway/latest/developerguide/api-gateway-payload-encodings-configure-with-control-service-api.html
# https://docs.aws.amazon.com/apigateway/latest/developerguide/api-gateway-payload-encodings-configure-with-console.html
# https://docs.aws.amazon.com/apigateway/latest/developerguide/api-gateway-mapping-template-reference.html
import boto3
import io
from base64 import b64encode,b64decode
from io     import BytesIO
from PIL import Image, ImageDraw, ImageFont
endpoint = '<INSERT_ENDPOINT_NAME_HERE>'
runtime= boto3.client('runtime.sagemaker')

imgByteArr = io.BytesIO()

image_src.save(imgByteArr, format=image_src.format)
imgByteArr = imgByteArr.getvalue()

# Send image via InvokeEndpoint API
response = runtime.invoke_endpoint(EndpointName=endpoint, ContentType='application/x-image', Body=imgByteArr)
result = response['Body'].read().decode()
# print(result)

**Display Detectron2 Object Detections inference results**

In [ ]:
res = json.loads(result) # convert json string to Python dict for parsing
im_out = object_visualizer(image_src, res, all_classed_list, True)
im_out.show()

## 7. Call Object Dection API using Python Request library

**Create AWS API gateway before performing this step**

In [ ]:
# import some common libraries
# Using Python Request library
import requests
import json
import numpy as np
import time
import io
from PIL import Image, ImageDraw, ImageFont

# Define Constants
API_INVOKE_URL='<INSERT_API_INVOKE_URL_HERE>'

# define variables
url=API_INVOKE_URL

def cloud_api_predict(headers, payload):
    # send POST request to url
    return requests.request("POST", url, headers=headers, data=payload).text

# Read image into memory
with open(input_image, 'rb') as f:
    payload = f.read()

headers = {
  'Accept': 'image/jpeg',
  'Content-Type': 'image/jpeg'
}

predictions=cloud_api_predict(headers, payload)

**Display Detectron2 Object Detections inference results from API**

In [ ]:
res = json.loads(predictions) # convert json string to Python dict for parsing
im_out = object_visualizer(image_src, res, all_classed_list, True)
im_out.show()